In [2]:
import io
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
import os
import sys
import json
from tqdm import tqdm

In [3]:
base_path = "E:/DS5500_Project/"

data_path = os.path.abspath('E:/DS5500_Project/features_v2.csv')
data = pd.read_csv(data_path)
   
# data.drop(columns=["Unnamed: 0"], inplace=True)


In [4]:
data

,session,trial,filename,crest_factor_F1,dasd_F1,diffvar_F1,form_factor_F1,iemg_F1,kurtosis_F1,kurtosis_f_F1,...,rms_W9,skew_W9,skew_f_W9,ssi_W9,sum_f_W9,var_f_W9,wflen_W9,willison_W9,gesture,participant
0,1,1,E:\DS5500-project\data\gesture-recognition-and...,7.091099,0.021834,0.000606,-6502.754507,365.456658,2.574033,85.547962,...,0.022588,-0.148877,5.529415,5.224663,0.109574,2.901826e-10,69.944068,15.0,10,1
1,1,2,E:\DS5500-project\data\gesture-recognition-and...,13.324514,0.023852,0.000689,1352.897349,379.664639,16.942218,329.114870,...,0.016172,-0.205427,9.332349,2.677940,0.077566,2.769772e-10,54.530919,23.0,10,1
2,1,3,E:\DS5500-project\data\gesture-recognition-and...,8.047954,0.016213,0.000326,1295.052394,296.823135,3.795827,134.751765,...,0.017039,-0.204604,9.078430,2.973002,0.064458,1.426359e-10,57.815909,13.0,10,1
3,1,4,E:\DS5500-project\data\gesture-recognition-and...,8.459555,0.026572,0.000864,14453.068135,479.688214,6.938354,98.832790,...,0.031982,-0.266818,5.067522,10.473811,0.131909,4.509337e-10,83.441138,0.0,10,1
4,1,5,E:\DS5500-project\data\gesture-recognition-and...,8.329011,0.026896,0.000901,11827.310277,464.350257,2.379207,116.412205,...,0.031134,-0.092205,5.414199,9.925787,0.177516,9.577100e-10,94.393506,3.0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,3,3,E:\DS5500-project\data\gesture-recognition-and...,7.771453,0.013208,0.000208,2599.892725,274.024844,2.176596,28.888640,...,0.020526,-0.293608,3.830298,4.314177,0.035283,1.966555e-11,40.970398,2.0,9,9
15347,3,4,E:\DS5500-project\data\gesture-recognition-and...,9.172642,0.019135,0.000429,1255.242161,385.695263,4.615531,133.237150,...,0.025825,-0.389927,4.200155,6.829604,0.076162,1.003953e-10,52.342457,0.0,9,9
15348,3,5,E:\DS5500-project\data\gesture-recognition-and...,5.923115,0.018533,0.000408,1713.228290,389.039490,1.912226,22.613382,...,0.027460,-0.321020,4.375813,7.721463,0.081006,1.125954e-10,59.843660,0.0,9,9
15349,3,6,E:\DS5500-project\data\gesture-recognition-and...,6.466362,0.015749,0.000292,-992.810528,319.503197,2.189195,21.864578,...,0.013359,-0.493375,3.832319,1.827333,0.062739,6.332731e-11,35.905471,0.0,9,9


In [5]:
def participant_expt(data, base_path, model_name, model):
    feature_cols = [c for c in data.columns if "_" in c]
    participants = list(range(1, 44))

    results = {}
    WINDOW = 5

    for i in tqdm(range(len(participants)-WINDOW + 1)):
        train_parts = participants[:i] + participants[i+WINDOW+1:]
        test_parts = participants[i:i+WINDOW]
        
        train_df = data[data['participant'].isin(train_parts)]
        test_df = data[data['participant'].isin(test_parts)]
        
        x_train = train_df.loc[:, feature_cols].values
        y_train = (train_df.loc[:, 'gesture'] - 1).values

        x_test = test_df.loc[:, feature_cols].values
        y_test = (test_df.loc[:, 'gesture'] - 1).values
        scaler = StandardScaler()

        scaler.fit(np.vstack([x_train, x_test]))

        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)

        model.fit(x_train, y_train)

        y_pred = model.predict(x_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision_micro = precision_score(y_test, y_pred, average='micro')
        precision_macro = precision_score(y_test, y_pred, average='macro')
        recall_micro = recall_score(y_test, y_pred, average='micro')
        recall_macro = recall_score(y_test, y_pred, average='macro')
        f1_micro = f1_score(y_test, y_pred, average='micro')
        f1_macro = f1_score(y_test, y_pred, average='macro')
        
        metrics_dict = {
            'Accuracy': accuracy,
            'Precision (Micro)': precision_micro,
            'Precision (Macro)': precision_macro,
            'Recall (Micro)': recall_micro,
            'Recall (Macro)': recall_macro,
            'F1 (Micro)': f1_micro,
            'F1 (Macro)': f1_macro
        }
        
        test_parts = [str(i) for i in test_parts]
        
        print(metrics_dict)

        results[" ".join(test_parts)] = metrics_dict
        
        
    with open(base_path + "/part_test_5" + model_name + ".json", "w") as file:
        json.dump(results, file)

In [6]:
lr_model = LogisticRegression(max_iter=5000, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
ensemble_model = VotingClassifier(estimators=[('rf', rf_model), ('lr', lr_model)], voting='soft')

In [7]:
participant_expt(data, base_path, "LR", lr_model)

  3%|▎         | 1/39 [00:56<35:52, 56.65s/it]

{'Accuracy': 0.715406162464986, 'Precision (Micro)': 0.715406162464986, 'Precision (Macro)': 0.7303661298782087, 'Recall (Micro)': 0.715406162464986, 'Recall (Macro)': 0.7154061624649861, 'F1 (Micro)': 0.7154061624649859, 'F1 (Macro)': 0.7152913280651945}


  5%|▌         | 2/39 [01:55<35:41, 57.87s/it]

{'Accuracy': 0.6750700280112045, 'Precision (Micro)': 0.6750700280112045, 'Precision (Macro)': 0.7002988730425347, 'Recall (Micro)': 0.6750700280112045, 'Recall (Macro)': 0.6750700280112045, 'F1 (Micro)': 0.6750700280112045, 'F1 (Macro)': 0.6691855833735076}


  8%|▊         | 3/39 [02:47<33:12, 55.34s/it]

...

{'Accuracy': 0.8061624649859944, 'Precision (Micro)': 0.8061624649859944, 'Precision (Macro)': 0.8171556249788209, 'Recall (Micro)': 0.8061624649859944, 'Recall (Macro)': 0.8061624649859944, 'F1 (Micro)': 0.8061624649859945, 'F1 (Macro)': 0.8069503250846543}


 97%|█████████▋| 38/39 [52:27<01:05, 65.64s/it]

{'Accuracy': 0.7955182072829131, 'Precision (Micro)': 0.7955182072829131, 'Precision (Macro)': 0.8092851926658716, 'Recall (Micro)': 0.7955182072829131, 'Recall (Macro)': 0.7955182072829131, 'F1 (Micro)': 0.7955182072829131, 'F1 (Macro)': 0.7964978488925701}


100%|██████████| 39/39 [53:25<00:00, 82.19s/it]

{'Accuracy': 0.7675070028011205, 'Precision (Micro)': 0.7675070028011205, 'Precision (Macro)': 0.7832824785227127, 'Recall (Micro)': 0.7675070028011205, 'Recall (Macro)': 0.7675070028011204, 'F1 (Micro)': 0.7675070028011205, 'F1 (Macro)': 0.7659908243601231}


In [8]:
participant_expt(data, base_path, "RF", rf_model)

  3%|▎         | 1/39 [00:33<21:14, 33.53s/it]

{'Accuracy': 0.6050420168067226, 'Precision (Micro)': 0.6050420168067226, 'Precision (Macro)': 0.6262291431452462, 'Recall (Micro)': 0.6050420168067226, 'Recall (Macro)': 0.6050420168067228, 'F1 (Micro)': 0.6050420168067226, 'F1 (Macro)': 0.5985777852996812}


  5%|▌         | 2/39 [01:06<20:26, 33.15s/it]

{'Accuracy': 0.6072829131652661, 'Precision (Micro)': 0.6072829131652661, 'Precision (Macro)': 0.6225207953719502, 'Recall (Micro)': 0.6072829131652661, 'Recall (Macro)': 0.6072829131652661, 'F1 (Micro)': 0.6072829131652661, 'F1 (Macro)': 0.5945595454042386}


  8%|▊         | 3/39 [01:39<19:44, 32.91s/it]

...

{'Accuracy': 0.6834733893557423, 'Precision (Micro)': 0.6834733893557423, 'Precision (Macro)': 0.6974901133315179, 'Recall (Micro)': 0.6834733893557423, 'Recall (Macro)': 0.6834733893557423, 'F1 (Micro)': 0.6834733893557423, 'F1 (Macro)': 0.6797109646234578}


 97%|█████████▋| 38/39 [21:46<00:39, 39.41s/it]

{'Accuracy': 0.6946778711484594, 'Precision (Micro)': 0.6946778711484594, 'Precision (Macro)': 0.7073623387940661, 'Recall (Micro)': 0.6946778711484594, 'Recall (Macro)': 0.6946778711484595, 'F1 (Micro)': 0.6946778711484594, 'F1 (Macro)': 0.6877695540620121}


100%|██████████| 39/39 [22:28<00:00, 34.59s/it]

{'Accuracy': 0.657703081232493, 'Precision (Micro)': 0.657703081232493, 'Precision (Macro)': 0.6705954658753309, 'Recall (Micro)': 0.657703081232493, 'Recall (Macro)': 0.657703081232493, 'F1 (Micro)': 0.657703081232493, 'F1 (Macro)': 0.6515325066510259}


In [9]:
participant_expt(data, base_path, "Ensemble", ensemble_model)

  3%|▎         | 1/39 [01:40<1:03:43, 100.63s/it]

{'Accuracy': 0.7126050420168067, 'Precision (Micro)': 0.7126050420168067, 'Precision (Macro)': 0.7287393297435816, 'Recall (Micro)': 0.7126050420168067, 'Recall (Macro)': 0.7126050420168069, 'F1 (Micro)': 0.7126050420168069, 'F1 (Macro)': 0.7122762786692516}


  5%|▌         | 2/39 [03:25<1:03:41, 103.29s/it]

{'Accuracy': 0.6801120448179272, 'Precision (Micro)': 0.6801120448179272, 'Precision (Macro)': 0.7072884134758957, 'Recall (Micro)': 0.6801120448179272, 'Recall (Macro)': 0.6801120448179272, 'F1 (Micro)': 0.6801120448179272, 'F1 (Macro)': 0.6743302595527577}


  8%|▊         | 3/39 [05:03<1:00:33, 100.92s/it]

...

{'Accuracy': 0.8123249299719888, 'Precision (Micro)': 0.8123249299719888, 'Precision (Macro)': 0.8237505997402972, 'Recall (Micro)': 0.8123249299719888, 'Recall (Macro)': 0.8123249299719888, 'F1 (Micro)': 0.8123249299719888, 'F1 (Macro)': 0.8132461754315055}


 97%|█████████▋| 38/39 [1:01:56<01:34, 94.85s/it]

{'Accuracy': 0.8028011204481793, 'Precision (Micro)': 0.8028011204481793, 'Precision (Macro)': 0.816416922396521, 'Recall (Micro)': 0.8028011204481793, 'Recall (Macro)': 0.8028011204481792, 'F1 (Micro)': 0.8028011204481793, 'F1 (Macro)': 0.8038497438129985}


100%|██████████| 39/39 [1:03:31<00:00, 97.72s/it]

{'Accuracy': 0.7708683473389356, 'Precision (Micro)': 0.7708683473389356, 'Precision (Macro)': 0.7873084828806464, 'Recall (Micro)': 0.7708683473389356, 'Recall (Macro)': 0.7708683473389355, 'F1 (Micro)': 0.7708683473389356, 'F1 (Macro)': 0.7697853285425336}
